In [1]:
import yfinance as yf
import pickle
import numpy as np
import pandas as pd

# -----------------------------------
# List of Top 10 Indian Stocks
# -----------------------------------
stocks = [
    "RELIANCE.NS", "HDFCBANK.NS", "TCS.NS", "INFY.NS",
    "ICICIBANK.NS", "SBIN.NS", "BHARTIARTL.NS",
    "BAJFINANCE.NS", "HINDUNILVR.NS", "LT.NS"
]

for ticker in stocks:
    print(f"\n📊 Predicting signal for {ticker}")

    # -----------------------------------
    # Load saved model files
    # -----------------------------------
    model = pickle.load(open(f"{ticker}_model.pkl", "rb"))
    scaler = pickle.load(open(f"{ticker}_scaler.pkl", "rb"))
    features = pickle.load(open(f"{ticker}_features.pkl", "rb"))

    # -----------------------------------
    # Fetch recent data
    # -----------------------------------
    data = yf.download(ticker, period="3mo")

    if data.empty:
        print("⚠ No recent data available")
        continue

    # -----------------------------------
    # Feature Engineering (same as training)
    # -----------------------------------
    data['Return'] = data['Close'].pct_change()
    data['MA20'] = data['Close'].rolling(20).mean()
    data['MA50'] = data['Close'].rolling(50).mean()
    data['Volatility'] = data['Return'].rolling(20).std()
    data['Volume_Change'] = data['Volume'].pct_change()
    data['MA_Cross'] = (data['MA20'] > data['MA50']).astype(int)

    delta = data['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # -----------------------------------
    # Data Cleaning
    # -----------------------------------
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(inplace=True)

    # -----------------------------------
    # Prediction
    # -----------------------------------
    X_latest = data[features].iloc[-1:]
    X_scaled = scaler.transform(X_latest)

    signal = model.predict(X_scaled)[0]

    # -----------------------------------
    # Human-readable output
    # -----------------------------------
    if signal == 1:
        print("📈 BUY Signal")
    elif signal == -1:
        print("📉 SELL Signal")
    else:
        print("⏸ HOLD Signal")


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
data.to_csv("signals.csv")